### 05. using tools with llms

- ge en modell tillgång till funktioner för att lösa problem, exempelvis matematik, privat data, dagsfärsk data

- todo: rita ett sekvensdiagram som beskriver flödet

In [52]:
import os
from datetime import datetime
import json

from dotenv import load_dotenv
#from pydantic import BaseModel

from google import genai # client for google gemini api
from google.genai import types
import httpx

load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY") # load the api key from another file (.env)

client = genai.Client(api_key=API_KEY) # create an instance of the client

In [53]:
promt = f"What is the current ({datetime.today()}) temperature in Amsterdam?"

In [54]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=promt,
)

print(response.candidates[0].content.parts[0].text)

I am sorry, I do not have access to real-time weather data. To find the current temperature in Amsterdam, I recommend checking a reliable weather app, website, or your local news channel.



In [55]:
# get coordinate from location name

def get_location_coordinates(location: str) -> dict:
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={location}&key={API_KEY}"

    response = httpx.get(url)
    result = json.loads(response.text)
    location = result["results"][0]["geometry"]["location"]

    return {"latitude": location["lat"], "longitude": location["lng"]}

In [56]:
# get temperature for location
def get_current_temperature(location: str) -> dict:

    coordiates = get_location_coordinates(location)

    url = f"https://weather.googleapis.com/v1/currentConditions:lookup?key={API_KEY}&location.latitude={coordiates["latitude"]}&location.longitude={coordiates["longitude"]}"
    response = httpx.get(url)
    result = json.loads(response.text)

    return {"temperature": result["temperature"]["degrees"], "unit": result["temperature"]["unit"]}


In [57]:
get_current_temperature_tool = {
    "name": "get_current_temperature",
    "description": "returns the current temperature for a given location",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {"type": "string", "description": "name of location"},
        },
        "required": ["location"]
    },
}

In [58]:
temp_tool = types.Tool(function_declarations=[get_current_temperature_tool])
config = types.GenerateContentConfig(tools=[temp_tool])

In [59]:
# when doing multiple calls to complete a task we need to keep the chat history
contents = [types.Content(role="user", parts=[types.Part(text=promt)])]

In [60]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=contents,
    config=config
)

if response.candidates[0].content.parts[0].function_call:
    tool_to_call = response.candidates[0].content.parts[0].function_call

    if tool_to_call.name == "get_current_temperature":
        tool_result = get_current_temperature(**tool_to_call.args)

        # add model call to tool chat history
        contents.append(types.Content(role="model", parts=[types.Part(function_call=tool_to_call)]))

        tool_result_part = types.Part.from_function_response(name=tool_to_call.name, response={"result": tool_result})

        # add tool call result to chat history
        contents.append(types.Content(role="user", parts=[tool_result_part]))

        # make new call to model with chat history
        response = client.models.generate_content(model="gemini-2.0-flash", contents=contents, config=config)

        print(response.text)


The current temperature in Amsterdam is 13.8 degrees Celsius.

